In [5]:
import pandas as pd
import json
import re
import os
import subprocess
import bibtexparser
from tqdm import tqdm
import re
from flair.data import Sentence
from flair.nn import Classifier

In [6]:
#TODO: write a function that normalizes the names of the authors from our annotations
# to fit the format generated by anystyle.io check below for examples and look for the pattern

In [7]:
def file_finder(file_str: str) -> str:
    """
    This function takes a file name and returns the path to the file in the all_data_articles.
    """
    title_doi = "../data/titles_doi.csv"
    folder_path = "../all_data_articles"
    
    #extract the doi from the file name
    doi = file_str.split("_")[-1].split(".")[0]

    # find the row in the csv file where the doi column ends with the doi
    df = pd.read_csv(title_doi)
    doi_row = df[df["DOI"].str.endswith(doi)]

    # extract the title from the row
    title_json = doi_row["Title"].values[0].replace(" ", "_") + ".json"

    for filename in os.listdir(folder_path):
        if filename.endswith(".json") and filename.startswith(title_json[:int(len(title_json)/3)]):
            return filename


In [8]:
def load_annotations(file_str: str) -> pd.DataFrame:
    """
    This function takes a file name and returns the annotations from the file.
    And also replaces missing values with None.
    """
    folder_path = "../data/annotated"

    file_path = os.path.join(folder_path, file_str)
    df = pd.read_excel(file_path)

    # replace missing values with None
    df = df.where(pd.notnull(df), None)

    # replace values marked with nan with None
    df = df.replace("nan", None)
    
    return df

In [9]:
def format_author_name(name):
    # TODO: probably need to handle more cases
    if name is None:
        return None
    if ' and ' in name:
        # Handle multiple authors
        authors = name.split(' and ')
        formatted_authors = [format_author_name(author) for author in authors]
        return ' and '.join(formatted_authors)
    else:
        parts = name.split()
        # Handle case where there is a middle initial
        if len(parts) == 3:
            return f"{parts[1]}, {parts[0]} {parts[2]}"
        # Handle case where there is no middle initial
        elif len(parts) == 2:
            return f"{parts[1]}, {parts[0]}"
        else:
            return name

In [10]:
def df_to_triplets(df: pd.DataFrame, format_author = True) -> set:
    """
    This function takes a dataframe and returns a set of triplets.
    """
    triplets = set()
    for i in range(len(df)):
        if format_author:
            triplet = (df.iloc[i]["Footnote"], format_author_name(df.iloc[i]["Authors"]), df.iloc[i]["Title"])
        else:
            triplet = (df.iloc[i]["Footnote"], df.iloc[i]["Authors"], df.iloc[i]["Title"])
        triplets.add(triplet)
    return triplets

In [11]:
def dict_to_triplets(extraction: dict) -> set:
    """
    Converts a dictionary of footnotes to a set of triplets
    """
    triplets = set()

    for number, references in extraction.items():
        for reference in references:
            author = reference[0]
            title = reference[1]

            if author == "":
                author = None

            if title == "":
                title = None
            
            triplets.add((int(number), author, title))

    return triplets

In [12]:
def information_extraction(file_path: str) -> set:
    """
    This function takes a file path and returns a set of triplets.
    """
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    extraction = {}

    prev_footnote = None

    for number, footnote in tqdm(article["footnotes"].items()):

        # If the footnote is ibid, use the previous footnote
        if footnote.startswith("Ibid"):
            footnote = prev_footnote

        # Store the footnote for the next iteration
        prev_footnote = footnote
        
        references = re.split(';',footnote)
        
        author_title_list = []

        for reference in references:

            command = ['ruby', 'anystyle.rb', str(reference)]
            bibtex = subprocess.run(command, stdout=subprocess.PIPE, text=True).stdout
            parsed_bibtex = bibtexparser.loads(bibtex).entries
            if parsed_bibtex:
                parsed_bibtex = parsed_bibtex[0]
            else:
                print(f"No valid BibTeX entry found in: {bibtex}, set to empty dict")
                parsed_bibtex = {}

            #print(parsed_bibtex)

            # Extract title, prioritizing 'booktitle' if both 'title' and 'booktitle' are present
            title = parsed_bibtex.get('booktitle', parsed_bibtex.get('title', None))

            # Extract author information
            author = parsed_bibtex.get('author', None)

            # Append author and title pair to the list
            author_title_list.append([author, title])

        # Store the list in the extraction dictionary with the footnote number as the key
        extraction[number] = author_title_list

    return dict_to_triplets(extraction)

In [13]:
def calculate_scores(triplets, extractions):
    TP = len(triplets & extractions)  # Intersection of triplets and extractions
    FP = len(extractions - triplets)  # Elements in extractions but not in triplets
    FN = len(triplets - extractions)  # Elements in triplets but not in extractions

    recall = TP / (TP + FN) if TP + FN != 0 else 0
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    return recall, precision, f_score

In [ ]:
# TODO: add some how to check for similarity between the authors and titles from the two sets. if for footnote number x 
# the authors and titles are very similar then we can assume that the extraction is correct.

def calculate_scores_adjusted(triplets: set, extraction: set) -> [float, float, float]:
    pass

In [22]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=True)
    extraction = information_extraction(title_json)
    recall, precision, f_score = calculate_scores(triplets, extraction)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")
    #break

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [01:14<00:00,  1.98it/s]



Recall: 0.03608247422680412
Precision: 0.03867403314917127
F-Score: 0.03733333333333333
--------------------------------------------------




In [47]:
# TODO: implement a simple approach with a tagger or just regrex to extract the author and title, and a simple split with ";"

def extract_citations(file_path: str) -> set:
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    
    citations = set()
    prev_footnote = None

    for footnote_number, footnote_text in tqdm(article["footnotes"].items()):
        # If the footnote is ibid, use the previous footnote
        if footnote_text.startswith("Ibid"):
            footnote_text = prev_footnote
        
        prev_footnote = footnote_text

        # Split the footnote into individual citations
        individual_citations = re.split(";", footnote_text)
        
        for citation_text in individual_citations:
            # Regular expression to extract authors and titles
            # TODO: try a better pattern
            pattern = re.compile(r'^(.+?),\s+(.+?)[,|(]')

            match = pattern.match(citation_text)
            
            if match:
                author = match.group(1)
                title = match.group(2)
                citations.add((int(footnote_number), author, title))


    return citations

 

In [49]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = extract_citations(title_json)
    recall, precision, f_score = calculate_scores(triplets, extraction)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")
    #break

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [00:00<00:00, 461529.36it/s]




Recall: 0.005154639175257732
Precision: 0.005780346820809248
F-Score: 0.005449591280653951
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [00:00<00:00, 347423.98it/s]




Recall: 0.15306122448979592
Precision: 0.189873417721519
F-Score: 0.1694915254237288
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.1382488479262673
Precision: 0.16042780748663102
F-Score: 0.1485148514851485
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx



100%|██████████| 115/115 [00:00<00:00, 338250.32it/s]




Recall: 0.009950248756218905
Precision: 0.011764705882352941
F-Score: 0.010781671159029648
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:00<00:00, 380608.35it/s]



Recall: 0.05106382978723404
Precision: 0.10526315789473684
F-Score: 0.06876790830945557
--------------------------------------------------




In [40]:
# TODO: write a function that extracts the author and title from the footnote using a tagger (i.e. flair)

def tagger_information_extraction(file_path: str) -> set:

    # load the NER tagger
    tagger = Classifier.load('ner-ontonotes-large')

    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    
    citations = set()
    prev_footnote = None

    for footnote_number, footnote_text in tqdm(article["footnotes"].items()):
        # If the footnote is ibid, use the previous footnote
        if footnote_text.startswith("Ibid"):
            footnote_text = prev_footnote
        
        prev_footnote = footnote_text

        # Split the footnote into individual citations
        individual_citations = re.split(";", footnote_text)

        for citation_text in individual_citations:
            
            author = None

            sentence = Sentence(citation_text)
            tagger.predict(sentence)
            for span in sentence.get_spans('ner'):
                if span.tag == "PERSON" or span.tag == "ORG":
                    if author is None:
                        author = span.text
                    else:
                        author += ("and " + span.text)
                if span.tag == "WORK_OF_ART":
                    citations.add((int(footnote_number), author, span.text))
                    author = None

                
    return citations


In [42]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = tagger_information_extraction(title_json)
    recall, precision, f_score = calculate_scores(triplets, extraction)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")

Labels - https___doi.org_10.1093_ehr_cew052.xlsx
2023-12-18 12:33:01,020 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


100%|██████████| 148/148 [00:19<00:00,  7.57it/s]




Recall: 0.24742268041237114
Precision: 0.22325581395348837
F-Score: 0.23471882640586797
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx
2023-12-18 12:33:35,719 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


100%|██████████| 124/124 [00:21<00:00,  5.88it/s]




Recall: 0.14795918367346939
Precision: 0.14285714285714285
F-Score: 0.14536340852130325
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx
2023-12-18 12:34:10,874 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


...



Recall: 0.2534562211981567
Precision: 0.291005291005291
F-Score: 0.27093596059113295
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx
2023-12-18 12:36:36,144 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


100%|██████████| 115/115 [00:23<00:00,  4.97it/s]




Recall: 0.13930348258706468
Precision: 0.15384615384615385
F-Score: 0.1462140992167102
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx
2023-12-18 12:37:13,227 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


100%|██████████| 100/100 [00:14<00:00,  6.71it/s]




Recall: 0.1148936170212766
Precision: 0.19148936170212766
F-Score: 0.14361702127659573
--------------------------------------------------




In [23]:
def extract_triplets(triplets):
    return list(filter(lambda triplet: 10 < triplet[0] < 20, triplets))

In [24]:
extract_triplets(extraction)

[(17, 'Hinton, J.', 'The Mass-Observers: A History, 1937–1949'),
 (18,
  'see, For and example, D.Dworkin',
  'Cultural Marxism in Postwar Britain: History, the New Left, and the Origins of Cultural Studies'),
 (17,
  'Hubble, N.',
  'Mass-Observation and Everyday Life: Culture, History, Theory'),
 (15,
  None,
  'For Young’s influence on various aspects of left thought, see the Contemporary British History special issue on his life and work'),
 (14,
  'Butler, L.',
  'Michael Young and the Institute of Community Studies: Family, Community and the Politics of Kinship’'),
 (19,
  'Harker, B.',
  '“The Trumpet of the Night”: Interwar Communists on BBC Radio’'),
 (13,
  'Butler, L.',
  'Michael Young and the Institute of Community Studies: Family, Community and the Politics of Kinship’'),
 (16, 'Jackson', 'Equality and the British Left'),
 (12, 'Francis', 'Ideas and Policies Under Labour'),
 (17,
  None,
  'For the best major works on Mass-Observation in these various contexts'),
 (11, 'B

In [25]:
extract_triplets(triplets)

[(14,
  'L. Butler',
  'Michael Young and the Institute of Community Studies: Family, Community and the Politics of Kinship'),
 (11, 'Brooke', 'Labour’s War'),
 (18,
  'D. Dworkin',
  'Cultural Marxism in Postwar Britain: History, the New Left, and the Origins of Cultural Studies'),
 (16, 'Jackson', 'Equality and the British Left'),
 (17, 'Highmore', 'Everyday Life and Cultural Theory: An Introduction'),
 (17, 'J. Hinton', 'The Mass-Observers: A History, 1937–1949'),
 (15, None, 'Contemporary British History'),
 (17,
  'N. Hubble',
  'Mass-Observation and Everyday Life: Culture, History, Theory'),
 (12, 'Francis', 'Ideas and Policies Under Labour'),
 (19,
  'B. Harker',
  '“The Trumpet of the Night”: Interwar Communists on BBC Radio'),
 (13,
  'L. Butler',
  'Michael Young and the Institute of Community Studies: Family, Community and the Politics of Kinship')]

In [ ]:
#(113, 'M. Young and P. Willmott', 'Family and Kinship in East London'),
#('113', 'Young, M. and Willmott, P.', 'Family and Kinship in East London'),